In [ ]:
import time 
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

Drive_path = '../chromedriver.exe'

s = Service(executable_path=Drive_path)

driver = webdriver.Chrome(service=s)

url = 'https://www.thegioididong.com/dtdd/iphone-15-pro-max/danh-gia'
all_scraped_comment = []
total_pages = 14

for page_num in range(1, total_pages + 1):
    current_url = url if page_num == 1 else f"{url}?p={page_num}"
    driver.get(current_url)
    
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "ul.comment-list li.par"))
        )
    except:
        print(f"Trang {page_num} không load được bình luận.")
        continue

    html_content = driver.page_source
    soup = BeautifulSoup(html_content, "html.parser")
    comments = soup.select("ul.comment-list li.par")
    
    print(f"Trang {page_num} tìm thấy {len(comments)} bình luận.")

    for c in comments:
        user = c.find("p", class_="cmt-top-name")
        text = c.find("p", class_="cmt-txt")
        rating = len(c.find_all("i", class_="iconcmt-starbuy"))

        if user and text:
            all_scraped_comment.append({
                "user": user.text.strip(),
                "comment_text": text.text.strip(),
                "rating": rating,
                "page_source": page_num
            })

driver.quit()
print(f"Tổng số bình luận: {len(all_scraped_comment)}")
print(all_scraped_comment[:5])

In [19]:
if len(all_scraped_comment) > 0:
    df = pd.DataFrame(all_scraped_comment)
    output_path = "../data/raw/reviews.csv"
    df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"Stored successfully!")

Stored successfully!
